<a href="https://colab.research.google.com/github/Selva-Subramanian/Zomato-restaurant-clustering-Sentiment-Analysis/blob/main/ZOMATO_RESTAURANT_CLUSTERING_AND_SENTIMENT_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**

Zomato is an Indian restaurant aggregator and food delivery start-up founded by Deepinder Goyal and Pankaj Chaddah in 2008. Zomato provides information, menus and user-reviews of restaurants, and also has food delivery options from partner restaurants in select cities.

India is quite famous for its diverse multi cuisine available in a large number of restaurants and hotel resorts, which is reminiscent of unity in diversity. Restaurant business in India is always evolving. More Indians are warming up to the idea of eating restaurant food whether by dining outside or getting food delivered. The growing number of restaurants in every state of India has been a motivation to inspect the data to get some insights, interesting facts and figures about the Indian food industry in each city. So, this project focuses on analysing the Zomato restaurant data for each city in India.

The Project focuses on Customers and Company, you have  to analyze the sentiments of the reviews given by the customer in the data and made some useful conclusion in the form of Visualizations. Also, cluster the zomato restaurants into different segments. The data is vizualized as it becomes easy to analyse data at instant. The Analysis also solve some of the business cases that can directly help the customers finding the Best restaurant in their locality and for the company to grow up and work on the fields they are currently lagging in.

This could help in clustering the restaurants into segments. Also the data has valuable information around cuisine and costing which can be used in cost vs. benefit analysis

Data could be used for sentiment analysis. Also the metadata of reviewers can be used for identifying the critics in the industry. 

# **Attribute Information**

## **Zomato Restaurant names and Metadata**
Use this dataset for clustering part

1. Name : Name of Restaurants

2. Links : URL Links of Restaurants

3. Cost : Per person estimated Cost of dining

4. Collection : Tagging of Restaurants w.r.t. Zomato categories

5. Cuisines : Cuisines served by Restaurants

6. Timings : Restaurant Timings

## **Zomato Restaurant reviews**
Merge this dataset with Names and Matadata and then use for sentiment analysis part

1. Restaurant : Name of the Restaurant

2. Reviewer : Name of the Reviewer

3. Review : Review Text

4. Rating : Rating Provided by Reviewer

5. MetaData : Reviewer Metadata - No. of Reviews and followers

6. Time: Date and Time of Review

7. Pictures : No. of pictures posted with review

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### <b> Importing the required libraries

In [271]:
import numpy as np # for numerical computation
import pandas as pd # for data manipulation
import math

%matplotlib inline
import matplotlib.pyplot as plt # for plotting visuals
import seaborn as sns
plt.style.use('seaborn')

from sklearn.cluster import KMeans 



In [335]:
# load the dataset
working_dir_path = '/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/CP4 - Unsupervised - Clustering/'
df1 = pd.read_csv(working_dir_path + 'Zomato Restaurant names and Metadata.csv')
df2 = pd.read_csv(working_dir_path + 'Zomato Restaurant reviews.csv')

### <b> Data Inspection

In [336]:
# sample of the dataset
df1.head()

,Name,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM
2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM"
3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM
4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no..."


In [337]:
# brief info of the dataset 
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         105 non-null    object
 1   Links        105 non-null    object
 2   Cost         105 non-null    object
 3   Collections  51 non-null     object
 4   Cuisines     105 non-null    object
 5   Timings      104 non-null    object
dtypes: object(6)
memory usage: 5.0+ KB


In [338]:
# brief description of the dataset
pd.set_option('display.max_rows', None)
df1.describe()

,Name,Links,Cost,Collections,Cuisines,Timings
count,105,105,105,51,105,104
unique,105,105,29,42,92,77
top,B-Dubs,https://www.zomato.com/hyderabad/the-glass-oni...,500,Food Hygiene Rated Restaurants in Hyderabad,"North Indian, Chinese",11 AM to 11 PM
freq,1,1,13,4,4,6


In [339]:
# columns in the datset
df1.columns

Index(['Name', 'Links', 'Cost', 'Collections', 'Cuisines', 'Timings'], dtype='object')

#### All restaurants have links and it is not much useful so let's just drop it. 


In [340]:
# Drop columns
df1 = df1.drop(['Links'], axis=1)

In [341]:
# replace null values with a string
df1 = df1.fillna('no info')
df1['Timings'][30] = 'no info'

In [342]:
# formatting the string in  Timings feature

word_list = [[['noon','Noon','PM'],['p.m.']],[['midnight','Midnight','AM'],['a.m.']]]
for i in word_list:
  j = i[1][0]
  for k in i[0]:
    df1['Timings'] = df1['Timings'].str.replace(k,j)

### <b> Exploratory Data Analysis

##### <b> Unique values in our features





In [343]:
# initialize empty lists
Unique_Collections = []
Unique_Cuisines = []

# features list
unique_list = [Unique_Collections, Unique_Cuisines]

# logic to extract unique values in the features
sum = 0
for i in df1.columns[2:-1]:
  l = unique_list[sum]
  sum += 1
  for j in range(len(df1[i])):
    for k in list(df1[i])[j].split(','):
      if k not in l and k[1:] not in l:
        if k[0]==' ':
          l.append(k[1:])
        else:
          l.append(k)

# number of unique items in the features
print(f"Total unique values in the feature 'Cost' is {len(df1['Cost'].value_counts())}\n")
for n,m in enumerate(df1.columns[2:-1]):
  print(f"Total unique values in the feature '{m}' is {len(unique_list[n])}\n")

Total unique values in the feature 'Cost' is 29

Total unique values in the feature 'Collections' is 38

Total unique values in the feature 'Cuisines' is 44



In [344]:
# count of unique items in the features

features = ['Collections', 'Cuisines']
unique_count = {}
k = {}
m = {}
for l in df1['Cost'].unique():
  m[l] = list(df1['Cost'].values).count(l)
unique_count['Cost'] = m
sum = 0
for i in unique_list:
  feature_list = []
  df1[features[sum]] = df1[features[sum]].apply(lambda x: x.split(','))
  df1[features[sum]].apply(lambda x: feature_list.extend(x))
  feature_list = [x[1:] if x[0]==' ' else x[:] for x in feature_list]
  for j in i:
    k[j] = feature_list.count(j)
  unique_count[features[sum]] = k
  sum += 1 
  

In [345]:
unique_count

{'Collections': {'American': 6,
  'Andhra': 6,
  'Arabian': 2,
  'Asian': 15,
  'BBQ': 2,
  'Bakery': 7,
  'Barbecue & Grill': 3,
  'Best Bakeries': 1,
  'Best Bars & Pubs': 4,
  'Best Milkshakes': 1,
  'Best shawarmas': 1,
  'Beverages': 5,
  'Biryani': 16,
  'Brilliant Biryanis': 1,
  'Burger': 5,
  'Cafe': 6,
  'Chinese': 43,
  'Continental': 21,
  'Corporate Favorites': 6,
  'Desserts': 13,
  'European': 4,
  'Fancy and Fun': 2,
  'Fast Food': 15,
  'Finger Food': 2,
  'Food Hygiene Rated Restaurants in Hyderabad': 8,
  'Frozen Delights': 1,
  'Function Venues': 2,
  'Gigs and Events': 3,
  'Goan': 1,
  'Gold Curated': 5,
  'Great Breakfasts': 2,
  'Great Buffets': 11,
  'Great Italian': 1,
  'Haleem': 1,
  'Happy Hours': 3,
  'Healthy Food': 1,
  "Hyderabad's Hottest": 7,
  'Hyderabadi': 4,
  'Ice Cream': 2,
  'Indonesian': 1,
  'Italian': 14,
  'Japanese': 2,
  'Juices': 1,
  'Kebab': 6,
  'Late Night Restaurants': 2,
  'Lebanese': 1,
  'Live Sports Screenings': 7,
  'Malaysian':

### <b> Feature Engineering

#### Clustering is based on similarity which is measured using distance, so it is necessary to have numerical(continuous) columns so as to quantify distance and compute similarity. Let's engineer such features. 

In [ ]:
# !pip install kmodes
# from kmodes.kmodes import KModes

In [ ]:
# # Elbow curve to find optimal K
# cost = []
# K = range(1,5)
# for num_clusters in list(K):
#     kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
#     kmode.fit_predict(x)
#     cost.append(kmode.cost_)
    
# plt.plot(K, cost, 'bx-')
# plt.xlabel('No. of clusters')
# plt.ylabel('Cost')
# plt.title('Elbow Method For Optimal k')
# plt.show()

In [ ]:
# x.insert(0, "Cluster", clusters, True)
# x

In [ ]:
# # Install gower
# !pip install gower

In [ ]:
# # Plot dendrogram
# import gower
# import scipy.cluster.hierarchy as sch
# plt.figure(figsize=(30,10))
# dendrogram = sch.dendrogram(sch.linkage(df1['Cost'], method = 'ward'))
# plt.title('Dendrogram')
# plt.xlabel('Restaurants')
# plt.ylabel('Euclidean Distances')
# plt.show()

In [ ]:
# df1['Cost'] = df1['Cost'].str.replace(',','')
# df1['Cost'] = df1['Cost'].astype(float)

In [ ]:
# X = df1['Cost']

In [ ]:
# X = np.array(X).reshape(-1,1)

In [ ]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=5)
# kmeans.fit(X)
# y_kmeans = kmeans.predict(X)

In [ ]:
# plt.figure(figsize=(20,5))
# plt.scatter(df1['Name'], X[:, 0], c=y_kmeans, s=50, cmap='viridis')

# # centers = kmeans.cluster_centers_
# # plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)